In [26]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [27]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries


# Data download
# ==============================================================================
#data = fetch_dataset(name="items_sales")
#data.to_parquet('items_sales.parquet', index=True)
data = pd.read_parquet('items_sales.parquet')
data = data.asfreq('D')
data.head()
exog = pd.DataFrame({
    'exog_1': np.random.normal(loc=0, scale=1, size=data.shape[0]),
    'exog_2': np.random.normal(loc=0, scale=1, size=data.shape[0]),
}, index=data.index)

end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()


# Create and train ForecasterAutoregMultiSeries
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 5,
                 encoding           = 'onehot',
                 transformer_series = StandardScaler(),
                 transformer_exog   = StandardScaler(),
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

forecaster.fit(series=data_train, exog=exog.loc[data_train.index])
forecaster

ForecasterAutoregMultiSeries 
Regressor: LGBMRegressor(random_state=123, verbose=-1) 
Lags: [1 2 3 4 5] 
Transformer for series: StandardScaler() 
Transformer for exog: StandardScaler() 
Series encoding: onehot 
Window size: 5 
Series levels (names): ['item_1', 'item_2', 'item_3'] 
Series weights: None 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['exog_1', 'exog_2'] 
Training range: ["'item_1': ['2012-01-01', '2014-07-15']", "'item_2': ['2012-01-01', '2014-07-15']", "'item_3': ['2012-01-01', '2014-07-15']"] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: boosting_type: gbdt, class_weight: None, colsample_bytree: 1.0, importance_type: split, learning_rate: 0.1, ... 
fit_kwargs: {} 
Creation date: 2024-08-21 22:18:47 
Last fit date: 2024-08-21 22:18:47 
Skforecast version: 0.13.0 
Python version: 3.12.4 
Forecas

In [28]:
levels = ['item_1', 'item_2', 'item_3']
n_levels = len(levels)
lags = np.array([1, 2, 3, 4, 5])
steps = 2
lags_shape = len(lags)
exog_shape = 2
encoding = 'onehot'
series_col_names = levels
encoding_mapping = {'item_1': 0, 'item_2': 1, 'item_3': 2}
# Exog es un diccionario donde las claves son el step y los son numpy arrays en los
# que cada fila es un level y cada columna es una variable exogena.
exog = {
    1:np.full(shape=(steps, exog_shape), fill_value=99, dtype=float),
    2:np.full(shape=(steps, exog_shape), fill_value=999, dtype=float),
}
exog

{1: array([[99., 99.],
        [99., 99.]]),
 2: array([[999., 999.],
        [999., 999.]])}

In [29]:
if encoding is not None:
    if encoding == 'onehot':
        levels_encoded = np.zeros((n_levels, len(series_col_names)), dtype=float)
        for idx, level in enumerate(levels):
            if level in series_col_names:
                levels_encoded[idx, series_col_names.index(level)] = 1.
    else:
        levels_encoded = np.array([encoding_mapping.get(level, None) for level in levels], dtype='float64').reshape(-1, 1)
    levels_encoded_shape = levels_encoded.shape[1]
else:
    levels_encoded_shape = 0
levels_encoded

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [30]:
last_window =  np.full(shape=(lags_shape, n_levels), fill_value=5, dtype=float)
predictions =  np.full(shape=(steps, n_levels), fill_value=np.nan, dtype=float)
print(last_window)
print(predictions)
last_window = np.concatenate((last_window, predictions), axis=0)
last_window

[[5. 5. 5.]
 [5. 5. 5.]
 [5. 5. 5.]
 [5. 5. 5.]
 [5. 5. 5.]]
[[nan nan nan]
 [nan nan nan]]


array([[ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [nan, nan, nan],
       [nan, nan, nan]])

In [31]:
features_shape = lags_shape + levels_encoded_shape + exog_shape
features = np.full(shape=(n_levels, features_shape), fill_value=np.nan, dtype=float)
features

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [32]:
if encoding is not None:
    features[:, lags_shape:lags_shape + levels_encoded_shape] = levels_encoded
features

array([[nan, nan, nan, nan, nan,  1.,  0.,  0., nan, nan],
       [nan, nan, nan, nan, nan,  0.,  1.,  0., nan, nan],
       [nan, nan, nan, nan, nan,  0.,  0.,  1., nan, nan]])

In [33]:
step = 1
i = 0
features[:, :lags_shape] = last_window[-lags - (steps - i), :].transpose()
features


array([[ 5.,  5.,  5.,  5.,  5.,  1.,  0.,  0., nan, nan],
       [ 5.,  5.,  5.,  5.,  5.,  0.,  1.,  0., nan, nan],
       [ 5.,  5.,  5.,  5.,  5.,  0.,  0.,  1., nan, nan]])

In [34]:
if exog is not None:
    features[:, -exog_shape:] = exog[step][i, ].transpose()
features

array([[ 5.,  5.,  5.,  5.,  5.,  1.,  0.,  0., 99., 99.],
       [ 5.,  5.,  5.,  5.,  5.,  0.,  1.,  0., 99., 99.],
       [ 5.,  5.,  5.,  5.,  5.,  0.,  0.,  1., 99., 99.]])

In [35]:
predictions

array([[nan, nan, nan],
       [nan, nan, nan]])

In [36]:
pred = forecaster.regressor.predict(features)
pred

array([1.16235447, 1.32205227, 1.15616982])

In [37]:
predictions[i, :] = pred
predictions

array([[1.16235447, 1.32205227, 1.15616982],
       [       nan,        nan,        nan]])

In [38]:
last_window

array([[ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [nan, nan, nan],
       [nan, nan, nan]])

In [39]:
last_window[-(steps - i), :] = pred
last_window

array([[5.        , 5.        , 5.        ],
       [5.        , 5.        , 5.        ],
       [5.        , 5.        , 5.        ],
       [5.        , 5.        , 5.        ],
       [5.        , 5.        , 5.        ],
       [1.16235447, 1.32205227, 1.15616982],
       [       nan,        nan,        nan]])

In [40]:
# data = fetch_dataset(name="items_sales")
# data.to_parquet('items_sales_2.parquet', index=True)
data = pd.read_parquet("items_sales_2.parquet")
data = data.asfreq("D")
data.head()
exog = pd.DataFrame(
    {
        "exog_1": np.random.normal(loc=0, scale=1, size=data.shape[0]),
        "exog_2": np.random.normal(loc=0, scale=1, size=data.shape[0]),
    },
    index=data.index,
)

end_train = "2014-07-15 23:59:00"
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

forecaster.fit(series=data_train, exog=exog.loc[data_train.index])

(last_window_values_dict, exog_values_dict, levels, prediction_index, _) = (
    forecaster._create_predict_inputs(
        steps=2, levels=None, last_window=None, exog=exog.loc[data_test.index]
    )
)

In [41]:
exog_values_dict

{'item_1': array([[-0.16900541, -0.15968014],
        [-0.13210193, -0.09496689]]),
 'item_2': array([[-0.16900541, -0.15968014],
        [-0.13210193, -0.09496689]]),
 'item_3': array([[-0.16900541, -0.15968014],
        [-0.13210193, -0.09496689]])}

In [42]:
exog_values = np.concat(list(exog_values_dict.values()))
exog_values

array([[-0.16900541, -0.15968014],
       [-0.13210193, -0.09496689],
       [-0.16900541, -0.15968014],
       [-0.13210193, -0.09496689],
       [-0.16900541, -0.15968014],
       [-0.13210193, -0.09496689]])

In [43]:
exog_values_dict_2 = {}
for i in range(steps):
    exog_values_dict_2[i+1] = exog_values[i::steps, :]
exog_values_dict_2
    

{1: array([[-0.16900541, -0.15968014],
        [-0.16900541, -0.15968014],
        [-0.16900541, -0.15968014]]),
 2: array([[-0.13210193, -0.09496689],
        [-0.13210193, -0.09496689],
        [-0.13210193, -0.09496689]])}

In [44]:
predictions_old = forecaster.predict(steps=50, exog=exog.loc[data_test.index])
predictions_new = forecaster.predict_new(steps=50, exog=exog.loc[data_test.index])
assert predictions_old.equals(predictions_new)

## Benchmark

In [51]:
n_series = 1000
n=365
index = pd.date_range(start='2021-01-01',periods=n, freq="D")
series = [pd.Series(np.random.normal(size=n), index=index, name=f"series_{i+1}") for i in range(n_series)]
data = pd.concat(series, axis=1)
print(f"Data shape: {data.shape}")


forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1, n_estimators=20),
                 lags               = 5,
                 encoding           = 'onehot',
                 transformer_series = StandardScaler(),
             )
forecaster.fit(series = data)

Data shape: (365, 1000)


In [63]:
import timeit

def benchmark_function():
    forecaster.predict(steps=5)
times = timeit.repeat(benchmark_function, repeat=5, number=1)
times = np.array(times)
print(f"Mean time: {times.mean()} , std: {times.std()}, max: {times.max()}, min: {times.min()}")

def benchmark_function():
    forecaster.predict_new(steps=5)
times = timeit.repeat(benchmark_function, repeat=5, number=1)
times = np.array(times)
print(f"Mean time: {times.mean()} , std: {times.std()}, max: {times.max()}, min: {times.min()}")

Mean time: 5.589584360003937 , std: 0.5060823248948181, max: 6.178835600032471, min: 4.852042399987113
Mean time: 1.8232917400076984 , std: 0.13196965044550993, max: 2.0850156000233255, min: 1.7260719999903813


In [64]:
assert forecaster.predict(steps=5).equals(forecaster.predict_new(steps=5))